In [57]:
import requests
import pandas as pd

# for stage in stages:
url = 'http://egw.ifsc-climbing.org/egw/ranking/json.php?'
competition_id = 7923
category = '5'
stage = 1

# Adding competition, category and route to url
url = url + 'comp=' + str(competition_id) + '&cat=' + category + '&route=' + str(stage)
comp_json = requests.get(url).json()

new_stage = []

for climber in comp_json['participants']:
    new_comp_stage = dict()
    # Add a dictionary with the information for each comp
    new_comp_stage.update({
        'compId': comp_json.get('WetId'),
        'category': climbing_category(comp_json.get('GrpId')),
        'stage': competition_stage(comp_json['route_order'])
    })

    new_comp_stage.update({
        'climberID': climber.get('PerId'),
        'startOrder': climber.get('start_order'),
        'rankAfterStage' : climber.get('result_rank'),
        'boulder1': climber.get('boulder1'),
        'boulder2': climber.get('boulder2'),
        'boulder3': climber.get('boulder3'),
        'boulder4': climber.get('boulder4'),
        'boulder5': climber.get('boulder5'),
    })
    new_stage.append(new_comp_stage)

# These ones should definitely be in the overall dataframe, not the climbers one    
#     'isoOpen': comp_json.get('route_iso_open'),
#     'isoClose': comp_json.get('route_iso_close'),
#     'stageStart': comp_json.get('route_start'),
#     'numProblems': comp_json.get('route_num_problems'),
#     'climbingTime': comp_json.get('route_climbing_time'),

new_stage

# comp_json['participants']
# new_comp_stage


[{'compId': '7923',
  'category': 'womenBouldering',
  'stage': '2. Qualification',
  'climberID': '11989',
  'startOrder': '1',
  'rankAfterStage': '1',
  'boulder1': 't2 z1',
  'boulder2': 't1 z1',
  'boulder3': 't1 z1',
  'boulder4': 't1 z1',
  'boulder5': 't3 z1'},
 {'compId': '7923',
  'category': 'womenBouldering',
  'stage': '2. Qualification',
  'climberID': '9989',
  'startOrder': '2',
  'rankAfterStage': '2',
  'boulder1': 't1 z1',
  'boulder2': 't2 z1',
  'boulder3': 't2 z2',
  'boulder4': 't1 z1',
  'boulder5': 'z1'},
 {'compId': '7923',
  'category': 'womenBouldering',
  'stage': '2. Qualification',
  'climberID': '9984',
  'startOrder': '22',
  'rankAfterStage': '3',
  'boulder1': 't1 z1',
  'boulder2': 't2 z1',
  'boulder3': 't2 z2',
  'boulder4': 't1 z1',
  'boulder5': 'z3'},
 {'compId': '7923',
  'category': 'womenBouldering',
  'stage': '2. Qualification',
  'climberID': '57446',
  'startOrder': '17',
  'rankAfterStage': '4',
  'boulder1': 't3 z1',
  'boulder2': 't2 z

In [20]:
def competition_stage(stage):
    stages = {
        '-1': 'General result',
        '0': '1. Qualification',
        '1': '2. Qualification',
        '2': 'Semi-Final',
        '3': 'Final'
    }
    return stages.get(stage, 'Other')

In [40]:
# TODO: Remove from here and inherit from py file
def climbing_category(cat):
    categories = {
        '1': 'menLead',
        '2': 'womenLead',
        '5': 'womenBouldering',
        '6': 'menBouldering',
        '23': 'menSpeed',
        '24': 'womenSpeed'}
    return categories.get(cat, 'otherCategory')